# Sewershed delineation

In this Jupyter notebook for Python, we delineate a sewershed based on a sewer network and census blocks.

## Data requirements

- Sewer mains
  * Sewer gravity mains and possibly force mains.
  * All flowing into one point (head of the sewershed).
  * The lines don't need to be spatially connected in the data, but the processing will assume that all lines flow into a single point.
- US Census blocks as polygons (areas) with demographics as attributes


<div class="alert alert-block alert-warning">
<b>Note:</b>
An experimental version of the v.dissolve tool available in <a href="https://github.com/OSGeo/grass/pull/2388">grass PR 2388</a> is needed.
</div>

## Software setup 

We will use a couple of standard Python packages and GRASS GIS.

For now, the setup here assumes Linux. Instructions for Windows are available at [GRASS GIS Jupyter notebooks wiki page](https://grasswiki.osgeo.org/wiki/GRASS_GIS_Jupyter_notebooks#Running_a_Jupyter_notebook_locally).

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
from pathlib import Path

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

## Get the data ready

This notebooks needs gravity mains (or all mains) as vector lines and census blocks as vector polygons with attributes. The gravity mains file should be in directory `data/sewers` and should be named `mains` with file extension appropriate for the format, e.g. `mains.shp`. Census blocks should be in `data/census` directory and named `blocks` plus a file extension. Either rename the files or modify the code below if needed.

The paths can be not only directories but also ZIP files. Similarly, the files can also be layers. The names will be passed to GDAL.

In [ ]:
sewers_directory = "data/sewers"
sewers_file = "mains"
census_directory = "data/census"
census_file = "blocks"

In [ ]:
grass_project = "data/sewershed_delineation"

To compute the data, we will use a GRASS project (aka location).

In [ ]:
!grass -e -c $sewers_directory $grass_project

In [ ]:
session = gj.init(grass_project)

We will use vector lines of gravity mains as our sewershed network. Here, we are using Raleigh as an example. We will also use the US 2020 census blocks for North Carolina. The census blocks are polygons (i.e., areas).

We store the names of GRASS vector maps in Python variables and will use the variables from now on.

In [ ]:
sewer_vector = "mains"
census_blocks_vector = "blocks"

In [ ]:
gs.run_command(
    "v.import", input=sewers_directory, layer=sewers_file, output=sewer_vector
)

To limit the number of census blocks we need to import, we set the computational region to the extent of sewer mains (extended in all directions by 100 map units - feet or meter).

In [ ]:
gs.run_command("g.region", vector=sewer_vector, res=1, grow=100)

Now we import the census blocks in the computational region.

In [ ]:
gs.run_command(
    "v.import",
    input=census_directory,
    layer=census_file,
    output=census_blocks_vector,
    extent="region",
)

In [ ]:
sewershed_pipes_map = gj.Map()
sewershed_pipes_map.d_vect(map=sewer_vector)
sewershed_pipes_map.show()

## Select census blocks

To get only the relevant census blocks, we select only census blocks which overlap with the sewer network.

In [ ]:
sewershed_blocks = "tmp_selected_blocks_for_sewershed"

gs.run_command(
    "v.select",
    ainput=census_blocks_vector,
    atype="area",
    binput=sewer_vector,
    output=sewershed_blocks,
    operator="intersects",
)

In [ ]:
census_blocks_map = gj.Map()
census_blocks_map.d_vect(map=sewershed_blocks)
census_blocks_map.show()

## Create sewershed

We still have only individual census blocks, now we connected them into one polygon (area) which will be our sewershed. We will do this by dissolving (removing) boundaries in between the census blocks. We will also compute aggregate statistics from several census block attribute columns to get population statistics for the sewershed.

Census column names need a key to explain the meaning, so we use shortened human-readable definitions for results.

In [ ]:
total_population_name = "P0010001"
# This list of tuples (pairs) linking new and old column names
population_columns = [
    ("race_white", "P0010003"),
    ("race_black_or_african_american", "P0010004"),
    ("race_american_indian_or_alaska_native", "P0010005"),
    ("race_asian", "P0010006"),
    ("race_native_hawaiian_and_other_pacific_islander", "P0010007"),
    ("race_some_other_race", "P0010008"),
    ("hispanic_or_latino", "P0020002"),
    ("population_18_years_and_over", "P0030001"),
    ("housing_units_total", "H0010001"),
    ("housing_occupied", "H0010002"),
    ("housing_vacant", "H0010003"),
    ("quarters_total", "P0050001"),
    ("quarters_correctional_facilities_for_adults", "P0050003"),
    ("quarters_nursing_facilities", "P0050005"),
    ("quarters_college_university_student_housing", "P0050008"),
    ("quarters_military_quarters", "P0050009"),
]

We will compute total population for the sewershed and express selected demographical information as a subset of that number. The SQL expression for aggregation as sum for total population is `sum(name)`, while the ratio is computed using `cast(sum(name) as real) / sum(total)`. The resulting column types are `integer` for the total population and `real` for the demographic ratios. Result columns need to be defined as `name type`. The following code takes care of both the expressions and the result column definitions.

In [ ]:
# Create expression sum(a).
aggregate_columns = [f"sum({total_population_name})"]
result_columns = ["total_population integer"]
float_type = "real"
for result_name, reference_name in population_columns:
    # Create expression sum(a) / sum(b).
    aggregate_columns.append(
        f"cast(sum({reference_name}) as {float_type}) "
        f"/ sum({total_population_name})"
    )
    result_columns.append(f"{result_name} {float_type}")

Finally, we need to select a column to use for dissolving. All census blocks with the same value in this column will be merged into one polygon and the boundaries in between will disappear.

We are using column State_Name because the state name is the same for all our census blocks and all our (previously selected) census blocks are part of the sewershed.

In [ ]:
# Dissolving is done based on the values in this column.
dissolve_column = "State_Name"  # from US census data

In [ ]:
sewershed = "sewershed_raleigh_sc"

# Dissolve selected blocks and compute aggregate statistics.
gs.run_command(
    "v.dissolve",
    input=sewershed_blocks,
    column=dissolve_column,
    output=sewershed,
    aggregate_column=aggregate_columns,
    result_column=result_columns,
)
# string concat county names during dissolve?
# v.db.select NC_blocks2020 columns="sum(P0010001)" where="County_Nam='Wake County'"

In [ ]:
sewer_map = gj.Map()
sewer_map.d_vect(map=sewershed)
sewer_map.show()

## Filling holes in the sewershed

If the sewershed has holes, we can fill them by removing the interior rings. The following removes all holes and standalone areas smaller than 1 square mile (2.788e+7 square feet, 2.6 km²).

In [ ]:
sewershed_filled_holes = "sewershed_raleigh_sc_filled_holes"

# Dissolve selected blocks and compute aggregate statistics.
gs.run_command(
    "v.clean",
    input=sewershed,
    output=sewershed_filled_holes,
    tool="rmarea",
    threshold=2.788e7,
)

In [ ]:
sewer_map = gj.Map()
sewer_map.d_vect(map=sewershed_filled_holes)
sewer_map.d_vect(map=sewer_vector)
sewer_map.show()

## Demographics

Using JSON, we will load sewershed attributes into a Pandas data frame.

In [ ]:
import json
import pandas as pd

data = pd.DataFrame(
    json.loads(
        gs.read_command("v.db.select", map=sewershed_filled_holes, format="json")
    )["records"]
)

In [ ]:
demographics = (
    data.drop("cat", axis=1)
    .drop("State_Name", axis=1)
    .transpose()
    .round(decimals=2)
    .rename(columns={0: "value"})
)

Inspect the demographics:

In [ ]:
demographics

## Save results to files

Save results in a file:

In [ ]:
output_file = (
    Path(sewers_directory) / "sewershed_with_demographics.shp"
)  # Modify as needed.
gs.run_command("v.out.ogr", input=sewershed_filled_holes, output=output_file)

In [ ]:
demographics_csv = Path(sewers_directory) / "demographics.csv"  # Modify as needed.
demographics.to_csv(demographics_csv, index_label="demographic")